![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
!pip install SeisMonitor
!pip install git+https://github.com/wayneweiqiang/GaMMA.git

In [1]:
import sys
repository_path = r"/home/emmanuel/SeisMonitor"  
sys.path.insert(0,repository_path)

**Please restart the kernel. It's mandatory to get everything up and running.**

Before starting to pick seismic phases, we need to download the data. Take a look in [Download](https://colab.research.google.com/github/ecastillot/SeisMonitor/blob/master/examples/1.download.ipynb). The purpose of this notebook is to pick seismic phases, then we are going to use a util function to download the data presented [here](https://colab.research.google.com/github/ecastillot/SeisMonitor/blob/master/examples/1.download.ipynb).

In [2]:
from utils4examples import quick_download
import os
download_folder = os.path.join(os.path.dirname(os.getcwd()),"out","downloads")
quick_download(out_folder)

01-18 10:11 [INFO] [SeisMonitor.metadata] running to create inventory and json files
01-18 10:11 [INFO] [SeisMonitor.metadata] Total time of execution: 0.224684 seconds
01-18 10:11 [INFO] [SeisMonitor.Downloader: True] ../out/downloads/CHI/CM.CHI.00.HHE__20191224T185959Z__20191224T205959Z.mseed  
01-18 10:11 [INFO] [SeisMonitor.Downloader: True] ../out/downloads/CHI/CM.CHI.00.HHN__20191224T185959Z__20191224T205959Z.mseed  
01-18 10:11 [INFO] [SeisMonitor.Downloader: True] ../out/downloads/CHI/CM.CHI.00.HHZ__20191224T185959Z__20191224T205959Z.mseed  
01-18 10:11 [INFO] [SeisMonitor.Downloader: True] ../out/downloads/PRA/CM.PRA.00.HHE__20191224T190000Z__20191224T210000Z.mseed  
01-18 10:11 [INFO] [SeisMonitor.Downloader: True] ../out/downloads/PRA/CM.PRA.00.HHN__20191224T190000Z__20191224T210000Z.mseed  
01-18 10:11 [INFO] [SeisMonitor.Downloader: True] ../out/downloads/PRA/CM.PRA.00.HHZ__20191224T190000Z__20191224T210000Z.mseed  
01-18 10:11 [INFO] [SeisMonitor.Downloader: True] ../out/

# Picking with EQTransformer

In [3]:
import os
from SeisMonitor.monitor.picker.ai import EQTransformer,EQTransformerObj
from SeisMonitor.monitor.picker import utils as piut

Using TensorFlow backend.


Cloning EQTransformer repository to use the original EQTransformer pretrained model

In [4]:
piut.git_clone_aipicker(name="EQTransformer",repo_dir=os.getcwd())

eqt_path = os.path.join(os.getcwd(),"EQTransformer")
if os.path.isdir(eqt_path):
    print(True)

There is alaready
True


We create the object: EQTransformerObj, which contains the EQTransformer parameters.

In [5]:
eqtobj = EQTransformerObj(eqt_path = eqt_path,
            n_processor = 6,
            overlap = 0.3,
            detection_threshold =0.1,
            P_threshold = 0.01,
            S_threshold = 0.01,
            batch_size = 20,
            number_of_plots = 0,
            plot_mode = 1 ) 

- EQTransformer is instanced with EQTransformerObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from EQTransformer
- We use eqt_picks_2_seismonitor_fmt to export seismic phases in specific format util for the rest of the workflow.

In [6]:
archive = os.path.join(os.path.dirname(os.getcwd()),"out")

mseed_storage = os.path.join(archive,"downloads")
json_dir = os.path.join(archive,"stations")
out_dir = os.path.join(archive,"picks","eqt")
result = os.path.join(archive,"picks","eqt","seismonitor_picks.csv")

eqt = EQTransformer(eqtobj)
eqt.pick(mseed_storage,json_dir,out_dir)
piut.eqt_picks_2_seismonitor_fmt(out_dir,mseed_storage,result)

01-18 10:11 [INFO] [SeisMonitor.Picker: EQTransfomer] Running EQTransformer
01-18 10:11 [INFO] [EQTransformer] Running EqTransformer  0.1.59
01-18 10:11 [INFO] [EQTransformer] *** Loading the model ...
2023-01-18 10:11:54.531068: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
2023-01-18 10:11:54.535542: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2592000000 Hz
2023-01-18 10:11:54.535893: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5635261b3740 executing computations on platform Host. Devices:
2023-01-18 10:11:54.535914: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
01-18 10:13 [INFO] [EQTransformer] *** Loading is complete!
01-18 10:13 [INFO] [EQTransformer] There are files for 7 stations in /home/emmanuel/SeisMonitor/out/downloads directory.
01-18 10:13 [INFO] [EQTransformer] Started worki

01-18 10:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 14.22 seconds.
01-18 10:13 [INFO] [EQTransformer] *** Detected: 62 events.
01-18 10:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/SeisMonitor/out/picks/eqt/results/CHI_outputs "
01-18 10:13 [INFO] [EQTransformer] Started working on FLO2, 2 out of 7 ...
01-18 10:13 [INFO] [EQTransformer] 20191224T190000Z__20191224T210000Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T210000Z__20191224T230000Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T230000Z__20191225T010000Z.mseed


01-18 10:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 6.92 seconds.
01-18 10:13 [INFO] [EQTransformer] *** Detected: 79 events.
01-18 10:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/SeisMonitor/out/picks/eqt/results/FLO2_outputs "
01-18 10:13 [INFO] [EQTransformer] Started working on GARC, 3 out of 7 ...
01-18 10:13 [INFO] [EQTransformer] 20191224T185959Z__20191224T205959Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T205959Z__20191224T225959Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T225959Z__20191225T005959Z.mseed


01-18 10:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 6.89 seconds.
01-18 10:13 [INFO] [EQTransformer] *** Detected: 89 events.
01-18 10:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/SeisMonitor/out/picks/eqt/results/GARC_outputs "
01-18 10:13 [INFO] [EQTransformer] Started working on ORTC, 4 out of 7 ...
01-18 10:13 [INFO] [EQTransformer] 20191224T185959Z__20191224T205959Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T205959Z__20191224T225959Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T225959Z__20191225T005959Z.mseed


01-18 10:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 7.23 seconds.
01-18 10:13 [INFO] [EQTransformer] *** Detected: 111 events.
01-18 10:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/SeisMonitor/out/picks/eqt/results/ORTC_outputs "
01-18 10:13 [INFO] [EQTransformer] Started working on PRA, 5 out of 7 ...
01-18 10:13 [INFO] [EQTransformer] 20191224T190000Z__20191224T210000Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T210000Z__20191224T230000Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T230000Z__20191225T010000Z.mseed


01-18 10:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 7.11 seconds.
01-18 10:13 [INFO] [EQTransformer] *** Detected: 236 events.
01-18 10:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/SeisMonitor/out/picks/eqt/results/PRA_outputs "
01-18 10:13 [INFO] [EQTransformer] Started working on URMC, 6 out of 7 ...
01-18 10:13 [INFO] [EQTransformer] 20191224T185959Z__20191224T205959Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T205959Z__20191224T225959Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T225959Z__20191225T005959Z.mseed


01-18 10:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 6.59 seconds.
01-18 10:13 [INFO] [EQTransformer] *** Detected: 355 events.
01-18 10:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/SeisMonitor/out/picks/eqt/results/URMC_outputs "
01-18 10:13 [INFO] [EQTransformer] Started working on YOT, 7 out of 7 ...
01-18 10:13 [INFO] [EQTransformer] 20191224T190000Z__20191224T210000Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T210000Z__20191224T230000Z.mseed
01-18 10:13 [INFO] [EQTransformer] 20191224T230000Z__20191225T010000Z.mseed


01-18 10:13 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 6.68 seconds.
01-18 10:13 [INFO] [EQTransformer] *** Detected: 71 events.
01-18 10:13 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/SeisMonitor/out/picks/eqt/results/YOT_outputs "
01-18 10:13 [INFO] [SeisMonitor.Picker: EQTransfomer] Total time of execution: 123.861072 seconds


,pick_id,arrival_time,probability,phasehint,network,station,location,instrument_type,author,creation_time,event_start_time,event_end_time,detection_probability,snr,station_lat,station_lon,station_elv,file_name
0,20191224.190236.10-P-CM.URMC.00.HH,2019-12-24 19:02:36.108393,0.14,P,CM,URMC,00,HH,EQTransformer,2023-01-18 10:13:58.487156,2019-12-24 19:02:35.568393,2019-12-24 19:02:41.858393,0.25,6.3,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/downloads/URMC/...
1,20191224.190240.25-S-CM.URMC.00.HH,2019-12-24 19:02:40.258393,0.18,S,CM,URMC,00,HH,EQTransformer,2023-01-18 10:13:58.564482,2019-12-24 19:02:35.568393,2019-12-24 19:02:41.858393,0.25,12.6,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/downloads/URMC/...
2,20191224.190243.92-P-CM.PRA.00.HH,2019-12-24 19:02:43.928393,0.74,P,CM,PRA,00,HH,EQTransformer,2023-01-18 10:13:58.487156,2019-12-24 19:02:43.788393,2019-12-24 19:02:58.178393,0.79,13.3,3.713833,-74.886167,457.0,/home/emmanuel/SeisMonitor/out/downloads/PRA/C...
3,20191224.190251.05-P-CM.ORTC.00.HH,2019-12-24 19:02:51.058393,0.51,P,CM,ORTC,00,HH,EQTransformer,2023-01-18 10:13:58.487156,2019-12-24 19:02:50.968393,2019-12-24 19:03:12.678393,0.93,2.2,3.909000,-75.246000,446.0,/home/emmanuel/SeisMonitor/out/downloads/ORTC/...
4,20191224.190253.98-S-CM.PRA.00.HH,2019-12-24 19:02:53.988393,0.63,S,CM,PRA,00,HH,EQTransformer,2023-01-18 10:13:58.564482,2019-12-24 19:02:43.788393,2019-12-24 19:02:58.178393,0.79,10.7,3.713833,-74.886167,457.0,/home/emmanuel/SeisMonitor/out/downloads/PRA/C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,20191225.005156.91-S-CM.URMC.00.HH,2019-12-25 00:51:56.918393,0.46,S,CM,URMC,00,HH,EQTransformer,2023-01-18 10:13:58.564482,2019-12-25 00:51:53.108393,2019-12-25 00:51:58.928393,0.84,14.4,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/downloads/URMC/...
1860,20191225.005201.71-P-CM.PRA.00.HH,2019-12-25 00:52:01.718393,0.55,P,CM,PRA,00,HH,EQTransformer,2023-01-18 10:13:58.487156,2019-12-25 00:52:01.808393,2019-12-25 00:52:16.148393,0.38,27.1,3.713833,-74.886167,457.0,/home/emmanuel/SeisMonitor/out/downloads/PRA/C...
1861,20191225.005206.58-P-CM.URMC.00.HH,2019-12-25 00:52:06.588393,0.16,P,CM,URMC,00,HH,EQTransformer,2023-01-18 10:13:58.487156,2019-12-25 00:52:05.838393,2019-12-25 00:52:12.208393,0.75,26.0,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/downloads/URMC/...
1862,20191225.005210.34-S-CM.URMC.00.HH,2019-12-25 00:52:10.348393,0.17,S,CM,URMC,00,HH,EQTransformer,2023-01-18 10:13:58.564482,2019-12-25 00:52:05.838393,2019-12-25 00:52:12.208393,0.75,14.4,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/downloads/URMC/...


# Picking with PhaseNet

In [7]:
import os
from SeisMonitor.monitor.picker.ai import PhaseNet,PhaseNetObj
from SeisMonitor.monitor.picker import utils as piut

Cloning PhaseNet repository to use the original PhaseNet pretrained model

In [8]:
piut.git_clone_aipicker(name="PhaseNet",repo_dir=os.getcwd())

pnet_path = os.path.join(os.getcwd(),"PhaseNet")
if os.path.isdir(pnet_path):
    print(True)

There is alaready
True


We create the object: PhaseNetObj, which contains the PhaseNet parameters.

In [9]:
pnetobj = PhaseNetObj(pnet_path,
            P_threshold=0.3, S_threshold=0.3,
            batch_size=100
            ) 

- PhaseNet is instanced with PhaseNetObj.
- We use **pick** method to pick seismic phases. The input parameters are:
    - Folder where is downloaded the data,
    - Folder where is downloaded the metadata 
    - Output folder from PhaseNet

In [10]:
archive = os.path.join(os.path.dirname(os.getcwd()),"out")
mseed_storage = os.path.join(archive,"downloads")
json_dir = os.path.join(archive,"stations")
out_dir = os.path.join(archive,"picks","pnet")
result = os.path.join(archive,"picks","pnet","seismonitor_picks.csv")

pnet = PhaseNet(pnetobj)
pnet.pick(mseed_storage,json_dir,out_dir)

01-18 10:13 [INFO] [SeisMonitor.Picker: PhaseNet] Moving mseed2onefolder in 0.01247 seconds.
01-18 10:13 [INFO] [SeisMonitor.Picker: PhaseNet] Running to create datalist
01-18 10:14 [INFO] [SeisMonitor.Picker: PhaseNet] Datalist created in 0.97962 seconds.
01-18 10:14 [INFO] [SeisMonitor.Picker: PhaseNet] Running PhaseNet pretrained model.
01-18 10:14 [INFO] [SeisMonitor.Picker: PhaseNet] Console-> python /home/emmanuel/SeisMonitor/examples/PhaseNet/run.py --mode=pred             --model_dir=/home/emmanuel/SeisMonitor/examples/PhaseNet/model/190703-214543 --data_dir=/home/emmanuel/SeisMonitor/out/picks/pnet/datadir             --data_list=/home/emmanuel/SeisMonitor/out/picks/pnet/datalist/fname.csv --output_dir=/home/emmanuel/SeisMonitor/out/picks/pnet/results            --batch_size=100 --tp_prob=0.3            --ts_prob=0.3 --resampling=-1 --input_mseed
01-18 10:14 [INFO] [SeisMonitor.Picker: PhaseNet] Running...
2023-01-18 10:14:02,933 Pred log: /home/emmanuel/SeisMonitor/out/picks/

waiting data_reader...





/home/emmanuel/SeisMonitor/out/picks/pnet/datadir/CM.PRA.00.HHE__20191224T210000Z__20191224T230000Z.mseed:  26%|██▋       | 127/480 [00:04<00:20, 16.93it/s]




/home/emmanuel/SeisMonitor/out/picks/pnet/datadir/CM.ORTC.00.HHN__20191224T205959Z__20191224T225959Z.mseed:  26%|██▌       | 124/480 [00:04<00:24, 14.61it/s]

/home/emmanuel/SeisMonitor/out/picks/pnet/datadir/CM.CHI.00.HHN__20191224T185959Z__20191224T205959Z.mseed:  26%|██▋       | 126/480 [00:04<00:25, 13.84it/s]



/home/emmanuel/SeisMonitor/out/picks/pnet/datadir/CM.PRA.00.HHZ__20191224T190000Z__20191224T210000Z.mseed:  27%|██▋       | 128/480 [00:04<00:24, 14.11it/s]






/home/emmanuel/SeisMonitor/out/picks/pnet/datadir/CM.URMC.00.HHE__20191224T205959Z__20191224T225959Z.mseed:  28%|██▊       | 136/480 [00:04<00:24, 14.08it/s]


/home/emmanuel/SeisMonitor/out/picks/pnet/datadir/CM.PRA.00.HHE__20191224T210000Z__20191224T230000Z.mseed:  28%|██▊       | 133/480 [00:04<00:16, 21.17it/s]
/home/emmanuel/SeisMonitor/out/picks/

Last batch: 40 samples
Done


01-18 10:20 [INFO] [PhaseNet: picks2df] Conversion pick_sample in pick_time ok. See your results in pick_df.csv.
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Single:0.5359241 >= Overlap:0.4885463 | Single selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Overlap selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Single:0.39485928 >= Overlap:0.3703616 | Single selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Overlap:0.71963 > Single:0.5523663 | Overlap selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Overlap:0.48246226 > Single:0.30613258 | Overlap selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Overlap selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Overlap:0.5154064 > Single:0.51133484 | Overlap selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Single:0.8491056 >= Overlap:0.7845322 | Single selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Single:0.831304 >= Overlap:0.8030857 | Single selected
01-18 10:20 [DEBUG] [PhaseNet: Select_picks] Single sel

,pick_id,arrival_time,probability,phasehint,network,station,location,instrument_type,author,creation_time,...,station_lat,station_lon,station_elv,file_name,mseed_start_time,mseed_end_time,sampling_rate,sample,segment,segment_type
0,20191224.190040.75-P-CM.GARC.0.HH,2019-12-24 19:00:40.758393,0.597702,P,CM,GARC,0,HH,PhaseNet,2023-01-18 15:18:42.087085,...,2.186667,-75.493167,1999.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998393,2019-12-24 20:59:59.998393,100.0,2576,723000,overlap
1,20191224.190046.64-P-CM.GARC.0.HH,2019-12-24 19:00:46.648394,0.489404,P,CM,GARC,0,HH,PhaseNet,2023-01-18 15:18:55.921758,...,2.186667,-75.493167,1999.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998394,2019-12-24 20:59:59.998394,100.0,165,726000,overlap
2,20191224.190051.62-S-CM.GARC.0.HH,2019-12-24 19:00:51.628394,0.379748,S,CM,GARC,0,HH,PhaseNet,2023-01-18 15:18:56.146434,...,2.186667,-75.493167,1999.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998394,2019-12-24 20:59:59.998394,100.0,663,726000,overlap
3,20191224.190051.62-S-CM.GARC.0.HH,2019-12-24 19:00:51.628394,0.402778,S,CM,GARC,0,HH,PhaseNet,2023-01-18 15:18:47.854939,...,2.186667,-75.493167,1999.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998394,2019-12-24 20:59:59.998394,100.0,2163,3000,single
4,20191224.190100.08-P-CM.GARC.0.HH,2019-12-24 19:01:00.088394,0.633422,P,CM,GARC,0,HH,PhaseNet,2023-01-18 15:18:47.953219,...,2.186667,-75.493167,1999.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 18:59:59.998394,2019-12-24 20:59:59.998394,100.0,9,6000,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16021,20191225.005953.18-P-CM.URMC.0.HH,2019-12-25 00:59:53.188393,0.955563,P,CM,URMC,0,HH,PhaseNet,2023-01-18 15:18:04.214073,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2319,717000,single
16022,20191225.005953.21-P-CM.URMC.0.HH,2019-12-25 00:59:53.218393,0.954471,P,CM,URMC,0,HH,PhaseNet,2023-01-18 15:19:57.970299,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2322,717000,single
16023,20191225.005956.67-S-CM.URMC.0.HH,2019-12-25 00:59:56.678393,0.649255,S,CM,URMC,0,HH,PhaseNet,2023-01-18 15:18:04.416889,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2668,717000,single
16024,20191225.005956.92-S-CM.URMC.0.HH,2019-12-25 00:59:56.928393,0.440276,S,CM,URMC,0,HH,PhaseNet,2023-01-18 15:19:40.191334,...,3.247190,-74.378790,725.0,/home/emmanuel/SeisMonitor/out/picks/pnet/data...,2019-12-24 22:59:59.998393,2019-12-25 00:59:59.998393,100.0,2693,717000,single
